In [1]:
from pathlib import Path

project_dir = Path('.').absolute().parent
model_dir = project_dir / 'logs/locomotion_baselines' / 'ppo' / 'A1GymEnv-v0_1'

In [2]:
import torch.nn as nn
from utils import ALGOS

model_path = model_dir / "A1GymEnv-v0.zip"
model = ALGOS["ppo"].load(model_path)

def extract_policy_layers(model):
    mlp_extractor = model.policy.mlp_extractor.policy_net
    action_net = model.policy.action_net

    layers = []
    for m in mlp_extractor.modules():
        if not isinstance(m, nn.Sequential):
            layers.append(m)
    layers.append(action_net)

    return nn.Sequential(*layers)

policy_net = extract_policy_layers(model)
policy_net.eval()

Sequential(
  (0): Linear(in_features=50, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=256, bias=True)
  (3): ReLU()
  (4): Linear(in_features=256, out_features=12, bias=True)
)

In [4]:
import torch
import numpy as np

# Export model weights as csv
params_dir = model_dir / 'parameters'
params_dir.mkdir(exist_ok=True, parents=True)

def save_tensor_as_csv(path, t: torch.Tensor):
    t_np = t.detach().cpu().numpy()
    np.savetxt(path, t_np, delimiter = ',')

for name, param in policy_net.named_parameters():
    if len(param.size()) == 1:
        param = torch.unsqueeze(param, axis=-1)
    name = name.replace('.', '_')
    print(name, param.size())
    save_tensor_as_csv(params_dir / f'{name}.csv', param)

0_weight torch.Size([256, 50])
0_bias torch.Size([256, 1])
2_weight torch.Size([256, 256])
2_bias torch.Size([256, 1])
4_weight torch.Size([12, 256])
4_bias torch.Size([12, 1])


In [5]:
# Export normalizer parameters
import pickle
normalizer_path = model_dir / "A1GymEnv-v0" / "vecnormalize.pkl"
with open(normalizer_path, "rb") as pkl:
    normalizer = pickle.load(pkl)

obs_mean = np.sqrt(normalizer.obs_rms.var + normalizer.epsilon)
obs_std = normalizer.obs_rms.mean 
obs_mean = obs_mean.reshape(1,-1)
obs_std = obs_std.reshape(1,-1)
print(obs_mean.shape, obs_std.shape)

np.savetxt(params_dir / 'obs_mean.csv', obs_mean, delimiter = ',')
np.savetxt(params_dir / 'obs_std.csv', obs_mean, delimiter = ',')

(1, 50) (1, 50)


In [6]:
# Export default pose and motor polarity
from blind_walking.envs.env_wrappers import simple_openloop

pose_offset = simple_openloop.LaikagoPoseOffsetGenerator()._pose.reshape(1,-1)
np.savetxt(params_dir / 'pose_offset.csv', pose_offset, delimiter = ',')
print(pose_offset)

motor_polarity = np.array([1, -1, -1] * 4).reshape(1,-1)
np.savetxt(params_dir / 'motor_signs.csv', motor_polarity, delimiter = ',')

[[ 0.    0.67 -1.25  0.    0.67 -1.25  0.    0.67 -1.25  0.    0.67 -1.25]]


/home/daniel/anaconda3/envs/rl-baselines-zoo-cu113/lib/python3.8/site-packages/gym/logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


In [7]:
# Export nn observations and actions
nn_obs = np.load(str(model_dir / 'stats' / 'nn_observations.npy'))
nn_act = np.load(str(model_dir / 'stats' / 'nn_actions.npy'))
nn_obs = np.squeeze(nn_obs)
nn_act = np.squeeze(nn_act)
np.savetxt(str(params_dir / 'nn_observations.csv'), nn_obs,  delimiter = ',')
np.savetxt(str(params_dir / 'nn_actions.csv'), nn_act, delimiter = ',')

In [14]:
# Export motion capture
motor_position = np.load(str(model_dir / 'stats' / 'motor_position.npy'))
motor_position = np.squeeze(motor_position)
np.savetxt(str(params_dir / 'motor_position.csv'), motor_position,  delimiter = ',')

motor_velocity = np.load(str(model_dir / 'stats' / 'motor_velocity.npy'))
motor_velocity = np.squeeze(motor_velocity)
np.savetxt(str(params_dir / 'motor_velocity.csv'), motor_velocity,  delimiter = ',')

base_vel = np.load(str(model_dir / 'stats' / 'base_vel.npy'))
base_vel = np.squeeze(base_vel)
np.savetxt(str(params_dir / 'base_vel.csv'), base_vel,  delimiter = ',')

base_rpy = np.load(str(model_dir / 'stats' / 'base_rpy.npy'))
base_rpy = np.squeeze(base_rpy)
np.savetxt(str(params_dir / 'base_rpy.csv'), base_rpy,  delimiter = ',')

base_rpy_rate = np.load(str(model_dir / 'stats' / 'base_rpy_rate.npy'))
base_rpy_rate = np.squeeze(base_rpy_rate)
np.savetxt(str(params_dir / 'base_rpy_rate.csv'), base_rpy_rate,  delimiter = ',')

reference_foot_contact = np.load(str(model_dir / 'stats' / 'reference_foot_contact.npy'))
reference_foot_contact = np.squeeze(reference_foot_contact)
np.savetxt(str(params_dir / 'reference_foot_contact.csv'), reference_foot_contact, delimiter=',')

In [8]:
# Export sample in-out pairs
import json

policy_net.eval()
policy_net = policy_net.to(torch.device('cpu'))
sample_output_dir = model_dir / 'sample_inp_oup'
sample_output_dir.mkdir(exist_ok=True, parents=True)

sample_inputs = {
    'zeros': torch.zeros(1, 50),
    'ones': torch.ones(1, 50)
}
inp_oup_names = {}
for name, inp_value in sample_inputs.items():
    inp_name = name + '_in.csv'
    oup_name = name + '_out.csv'
    inp_oup_names[inp_name] = oup_name
    oup_value = policy_net(inp_value)
    save_tensor_as_csv(sample_output_dir / inp_name, inp_value)
    save_tensor_as_csv(sample_output_dir / oup_name, oup_value)

with open(sample_output_dir / 'inp_oup_name_pairs.txt', 'w') as file:
    for inp_name, oup_name in inp_oup_names.items():
        line = ','.join([inp_name, oup_name]) + "\n"
        file.write(line)

In [9]:
# Export butterworth filter coefficients and history
import gym 
import numpy as np
import utils.import_envs
env = gym.make("A1GymEnv-v0")
env.reset()

filter = env.robot._action_filter
a = filter.a.T.copy()
b = filter.b.T.copy()
print(a.shape, b.shape)
print(a)

np.savetxt(params_dir / 'filter_a_coeff.csv', a, delimiter =',')
np.savetxt(params_dir / 'filter_b_coeff.csv', b, delimiter =',')


pybullet build time: Jan 19 2022 23:07:37


TypeError: __init__() missing 1 required positional argument: 'gait_name'